In [ ]:
%load_ext autoreload
%autoreload 2
# %load_ext nb_black

In [ ]:
!pip install config

from google.colab import drive
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2

root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config
import pandas as pd
import csv

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

from utils.metrics import windowdiff, pk
from sklearn.metrics import precision_recall_fscore_support
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


In [ ]:
# import sys, os
# import config

# config.root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
# sys.path.insert(0, config.root_path)

In [ ]:
def perf_measure(y_actual, y_pred):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_pred)):
        if y_actual[i] == y_pred[i] == 1:
            TP += 1
        if y_pred[i] == 1 and y_actual[i] != y_pred[i]:
            FP += 1
        if y_actual[i] == y_pred[i] == 0:
            TN += 1
        if y_pred[i] == 0 and y_actual[i] != y_pred[i]:
            FN += 1
    return (TP, FP, TN, FN)


def get_precision(TP, FP, TN, FN):
    if TP == 0:
        return 0
    if TP + FP == 0:
        return 0
    return TP / (TP + FP)

def get_recall(TP, FP, TN, FN):
    if TP == 0:
        return 0
    if TP + FP == 0:
        return 0
    return TP / (TP + FN)

In [ ]:
# complex
fiction_tuple = ("fiction", "57488", "BfCoW", "fiction") # dataset from, num samples, hash, dataset to
clinical_tuple = ("clinical", "38575", "En2mj", "clinical")
wiki_tuple = ("wiki", "37412", "6YPov", "wiki")
wiki_to_clinical_tuple = ("wiki", "37412", "6YPov", "clinical")

In [ ]:
# hybrid conv
wiki_tuple = ("wiki", "40561", "hboPk", "wiki")
fiction_tuple = ("fiction", "61286", "8RliJ", "fiction")
clinical_tuple = ("clinical", "39968", "3KNFn", "clinical")

In [ ]:
model_type = "LDABERT2/hybrid-conv"
lda_gamma = 15

model_config = wiki_tuple

model_signature = "{}-{}-1-pct-1-aug_{}/no-finetune".format(
    *model_config
)

# predictions_path = "{}/predictions/{}/predictions/{}/{}/predictions.csv".format(
#     config.root_path, model_hash, model_type, model_signature
# )

# # this is for when we want to test trained datasets and how they infer on another dataset.
# predictions_path = "{}/predictions/{}_{}-from-{}-to_lda-{}/predictions.csv".format(
#     config.root_path, model_config[2], model_config[0], model_config[3], lda_gamma
# )

# new training log
predictions_path = "{}/models/LDABERT2/hybrid-conv/{}-{}-1-pct-1-aug_{}/finetune-True/checkpoint/training.log".format(
    config.root_path, model_config[0], model_config[1], model_config[2]
)

In [ ]:
predictions = []
reader = csv.DictReader(open(predictions_path, encoding="utf8"))
for row in reader:
    predictions.append(row)

In [ ]:
dict(predictions[0])

{'epoch': '0',
 'accuracy': '0.24117234349250793',
 'loss': '0.6884562969207764',
 'val_accuracy': '0.49993595480918884',
 'val_loss': '0.682073175907135'}

In [ ]:
from collections import OrderedDict

if type(predictions[0]) is OrderedDict:
  predictions = [dict(x) for x in predictions]
  
# predictions

In [ ]:
prediction_thresholds = [
    #     0.25,
    #     0.3,
    #     0.35,
    #     0.4,
    #     0.45,
    #     0.475,
    #     0.49, # conv
    #     0.495, # conv
    #     0.499, # conv
    0.5,
    #     0.6,
    0.75,
    #     0.85,
    #     0.9,
    0.95,
    0.98,
    #     0.99,
    #     0.995,
    #     0.999,
    0.9999,
]

In [ ]:
full_preds = []
for p in predictions:
    curr_preds = []
    for thresh in prediction_thresholds:
        curr_preds.append(1 if float(p["logit"]) > thresh else 0)
    full_preds.append([*curr_preds, p["ground_truth"]])
    print("predictions:", *curr_preds, "real:", p["ground_truth"])

columns = [*prediction_thresholds, "ground_truth"]
data = full_preds

KeyError: ignored

In [ ]:
df = pd.DataFrame(data, columns=columns)
df

,0.5,0.75,0.95,0.98,0.9999,ground_truth


In [ ]:
ground_truth = [int(k) for k in df["ground_truth"].tolist()]
new_predictions = []

for p_t in prediction_thresholds:
    new_predictions.append(df[p_t].tolist())

print("TP, FP, TN, FN")
df_data = []
string_ground_truth = "".join([str(s) for s in ground_truth])

# evaluation_window = 10
evaluation_windows = [
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
]  # [2, 5, 7, 10, 14]
for evaluation_window in evaluation_windows:
    for thresh, pred_set in zip(prediction_thresholds, new_predictions):
        string_pred_set = "".join([str(s) for s in pred_set])

        pk_result = pk(string_pred_set, string_ground_truth, evaluation_window)
        window_diff_result = windowdiff(
            string_pred_set, string_ground_truth, evaluation_window
        )
        
        TP, FP, TN, FN = perf_measure(ground_truth, pred_set)
        precision = get_precision(TP, FP, TN, FN)
        recall = get_recall(TP, FP, TN, FN)

        # create the dataframe
        df_data.append(
            [
                evaluation_window,
                thresh,
                TP, FP, TN, FN,
                precision,
                recall,
                pk_result,
                window_diff_result,
            ]
        )

        print(thresh, (TP, FP, TN, FN), precision, recall)

NameError: ignored

In [ ]:
df_evaluation_set = pd.DataFrame(
    df_data, columns=["K", "Threshold", "TP", "FP", "TN", "FN", "precision", "recall", "Pk", "WindowDiff"]
)
df_evaluation_set

,K,Threshold,TP,FP,TN,FN,precision,recall,Pk,WindowDiff
0,1,0.5000,10,361,1821,58,0.026954,0.147059,0.186222,0.186222
1,1,0.7500,0,3,2179,68,0.000000,0.000000,0.031556,0.031556
2,1,0.9500,0,0,2182,68,0.000000,0.000000,0.030222,0.030222
3,1,0.9800,0,0,2182,68,0.000000,0.000000,0.030222,0.030222
4,1,0.9999,0,0,2182,68,0.000000,0.000000,0.030222,0.030222
...,...,...,...,...,...,...,...,...,...,...
70,15,0.5000,10,361,1821,58,0.026954,0.147059,0.599284,0.837209
71,15,0.7500,0,3,2179,68,0.000000,0.000000,0.347048,0.349284
72,15,0.9500,0,0,2182,68,0.000000,0.000000,0.340340,0.340340
73,15,0.9800,0,0,2182,68,0.000000,0.000000,0.340340,0.340340


In [ ]:
print(df_evaluation_set.to_latex(index=False))

\begin{tabular}{rrrrrrrrrr}
\toprule
 K &  Threshold &  TP &  FP &   TN &  FN &  precision &   recall &       Pk &  WindowDiff \\
\midrule
 1 &     0.5000 &  10 & 361 & 1821 &  58 &   0.026954 & 0.147059 & 0.186222 &    0.186222 \\
 1 &     0.7500 &   0 &   3 & 2179 &  68 &   0.000000 & 0.000000 & 0.031556 &    0.031556 \\
 1 &     0.9500 &   0 &   0 & 2182 &  68 &   0.000000 & 0.000000 & 0.030222 &    0.030222 \\
 1 &     0.9800 &   0 &   0 & 2182 &  68 &   0.000000 & 0.000000 & 0.030222 &    0.030222 \\
 1 &     0.9999 &   0 &   0 & 2182 &  68 &   0.000000 & 0.000000 & 0.030222 &    0.030222 \\
 2 &     0.5000 &  10 & 361 & 1821 &  58 &   0.026954 & 0.147059 & 0.321476 &    0.324589 \\
 2 &     0.7500 &   0 &   3 & 2179 &  68 &   0.000000 & 0.000000 & 0.062250 &    0.062250 \\
 2 &     0.9500 &   0 &   0 & 2182 &  68 &   0.000000 & 0.000000 & 0.059582 &    0.059582 \\
 2 &     0.9800 &   0 &   0 & 2182 &  68 &   0.000000 & 0.000000 & 0.059582 &    0.059582 \\
 2 &     0.9999 &   0 & 

In [ ]:
evaluation_type = "WindowDiff" # "WindowDiff"
for thresh_set in range(len(prediction_thresholds)):
    print()
    for ind in df_evaluation_set.index:
        # print only the ones needed for creating graphs
        if (df_evaluation_set["K"][ind] in [2,5,7,10,14]):
            if (ind % len(prediction_thresholds)) + 1 == thresh_set + 1:
                print(
                    (df_evaluation_set["K"][ind], df_evaluation_set[evaluation_type][ind])
                )


(2, 0.3245887060915963)
(5, 0.5650044523597507)
(7, 0.6537433155080213)
(10, 0.7394020526550648)
(14, 0.8243182834152883)

(2, 0.062249888839484215)
(5, 0.14559216384683882)
(7, 0.19117647058823528)
(10, 0.2543507362784471)
(14, 0.33124720607957087)

(2, 0.059582036460649176)
(5, 0.13891362422083706)
(7, 0.18449197860962566)
(10, 0.24765729585006693)
(14, 0.32275368797496645)

(2, 0.059582036460649176)
(5, 0.13891362422083706)
(7, 0.18449197860962566)
(10, 0.24765729585006693)
(14, 0.32275368797496645)

(2, 0.059582036460649176)
(5, 0.13891362422083706)
(7, 0.18449197860962566)
(10, 0.24765729585006693)
(14, 0.32275368797496645)
